> **Rappel** : clique sur une cellule grise, puis **Shift + Entree** pour l'executer.
> Execute les cellules **dans l'ordre** de haut en bas.

La cellule suivante prepare les outils. **Tu n'as pas besoin de la lire**
\u2014 execute-la simplement avec Shift+Entree.

---

# Lecon 2 : Apprendre de ses erreurs

## Le secret de l'IA : se tromper, corriger, recommencer

Imagine que tu apprends a lancer une balle dans un panier :
1. Tu lances -> tu rates a droite
2. Tu corriges un peu a gauche
3. Tu relances -> plus pres !
4. Tu continues jusqu'a marquer

L'IA fait **exactement** pareil. Elle fait une prediction, regarde si c'est
bon, et ajuste. Ca s'appelle **l'entrainement**.

> **Vocabulaire de cette lecon**
> - **loss** : l'erreur du modele (plus c'est bas, mieux c'est)
> - **gradient** : la direction de correction (de combien ajuster)
> - **epoch** : un passage complet sur toutes les donnees
> - **entrainement** (*training*) : le processus d'apprentissage par repetition

In [ ]:
# ============================================================
# Cellule d'initialisation \u2014 execute sans lire (Shift+Entree)
# ============================================================

import uuid

from IPython.display import HTML, display

_exercices_faits = set()
_NB_TOTAL = 3


def verifier(num_exercice, condition, message_ok, message_aide=""):
    """Valide un exercice avec feedback HTML vert/rouge + compteur."""
    try:
        _result = bool(condition)
    except Exception:
        _result = False
    if _result:
        _exercices_faits.add(num_exercice)
        n = len(_exercices_faits)
        barre = "\U0001f7e9" * n + "\u2b1c" * (_NB_TOTAL - n)
        display(
            HTML(
                f'<div style="padding:10px;background:#d4edda;border-left:5px solid #28a745;'
                f'margin:8px 0;border-radius:4px;font-family:system-ui,-apple-system,sans-serif">'
                f"\u2705 <b>{message_ok}</b><br>"
                f'<span style="color:#555">Progression : {barre} {n}/{_NB_TOTAL}</span></div>'
            )
        )
        if n == _NB_TOTAL:
            display(
                HTML(
                    '<div style="padding:12px;background:linear-gradient(135deg,#3949ab,#6a1b9a);'
                    "color:white;border-radius:8px;text-align:center;font-family:system-ui,-apple-system,sans-serif;"
                    'font-size:1.2em;margin:8px 0">\U0001f3c6 <b>Bravo ! Toutes les activites de cette lecon sont terminees !</b></div>'
                )
            )
    else:
        display(
            HTML(
                f'<div style="padding:10px;background:#fff3cd;border-left:5px solid #ffc107;'
                f'margin:8px 0;border-radius:4px;font-family:system-ui,-apple-system,sans-serif">'
                f"\U0001f4a1 <b>{message_aide}</b></div>"
            )
        )


def exercice(numero, titre, consigne, observation=""):
    """Affiche la banniere d'exercice."""

    def _style_code(text):
        return text.replace(
            "<code>",
            '<code style="font-size:0.95em;background:#bbdefb;'
            'padding:1px 5px;border-radius:3px;font-family:monospace;">',
        )

    obs = ""
    if observation:
        obs = (
            f'<div style="margin-top:6px;color:#555;font-size:0.92em;">'
            f"<b>Ce que tu vas voir\u00a0:</b> {_style_code(observation)}</div>"
        )
    display(
        HTML(
            f'<div style="border-left:5px solid #1565c0;background:#e8f0fe;'
            f"padding:12px 16px; margin:4px 0 10px 0; border-radius:0 8px 8px 0;"
            f'font-family:system-ui,-apple-system,sans-serif; font-size:0.95em;">'
            f'<b style="color:#0d47a1;">Exercice\u00a0{numero} \u2014 {titre}</b><br>'
            f"{_style_code(consigne)}{obs}</div>"
        )
    )


def afficher_evolution_loss(pertes, titre="Courbe de loss"):
    """Courbe de loss animee (la ligne se dessine + hover = valeur)."""
    if not pertes:
        return
    uid = uuid.uuid4().hex[:8]
    n = len(pertes)
    max_loss = max(pertes)
    min_loss = min(pertes)
    # Dimensions du SVG
    w, h, pad = 500, 160, 30
    pw = w - 2 * pad  # largeur zone de dessin
    ph = h - 2 * pad  # hauteur zone de dessin
    # Construire le path SVG
    points = []
    circles = ""
    for i, loss in enumerate(pertes):
        x = pad + (i / max(n - 1, 1)) * pw
        y = pad + (1 - (loss - min_loss) / max(max_loss - min_loss, 1e-6)) * ph
        points.append(f"{x:.1f},{y:.1f}")
        circles += (
            f'<circle cx="{x:.1f}" cy="{y:.1f}" r="3" fill="#1565c0" opacity="0" '
            f'style="transition:opacity 0.2s"><title>Epoch {i}: {loss:.3f}</title></circle>'
        )
    path_d = "M" + " L".join(points)
    path_len = n * 20  # approximation
    # Axes
    axes = (
        f'<line x1="{pad}" y1="{pad}" x2="{pad}" y2="{h - pad}" stroke="#ccc" stroke-width="1"/>'
        f'<line x1="{pad}" y1="{h - pad}" x2="{w - pad}" y2="{h - pad}" stroke="#ccc" stroke-width="1"/>'
        f'<text x="{pad - 5}" y="{pad + 5}" text-anchor="end" font-size="10" fill="#999">{max_loss:.2f}</text>'
        f'<text x="{pad - 5}" y="{h - pad + 4}" text-anchor="end" font-size="10" fill="#999">{min_loss:.2f}</text>'
        f'<text x="{pad}" y="{h - pad + 16}" font-size="10" fill="#999">0</text>'
        f'<text x="{w - pad}" y="{h - pad + 16}" text-anchor="end" font-size="10" fill="#999">{n - 1}</text>'
    )
    display(
        HTML(
            f"<!-- tuto-viz -->"
            f'<div style="margin:8px 0"><b>{titre}</b>'
            f'<svg id="lc{uid}" width="{w}" height="{h}" style="margin-top:4px;display:block;background:#fafafa;border-radius:4px">'
            f"{axes}"
            f'<path d="{path_d}" fill="none" stroke="#1565c0" stroke-width="2" '
            f'stroke-dasharray="{path_len}" stroke-dashoffset="{path_len}" '
            f'style="animation:draw{uid} 1.5s ease forwards"/>'
            f"{circles}"
            f"</svg>"
            f"<style>@keyframes draw{uid}{{to{{stroke-dashoffset:0}}}}</style>"
            f"<script>(function(){{"
            f'var svg=document.getElementById("lc{uid}");'
            f'svg.addEventListener("mouseover",function(e){{'
            f'var c=e.target.closest("circle");if(c)c.setAttribute("opacity","1")}});'
            f'svg.addEventListener("mouseout",function(e){{'
            f'var c=e.target.closest("circle");if(c)c.setAttribute("opacity","0")}})'
            f"}})();</script>"
            f'<div style="color:#555;font-size:0.85em;margin-top:4px">'
            f"Survole les points pour voir la valeur exacte</div></div>"
        )
    )


print("Outils de visualisation charges !")

---
## Etape 1 : Mesurer l'erreur (la loss)

D'abord, il faut un moyen de dire **a quel point** le modele s'est trompe.
On appelle ca la **loss** (perte en anglais).

- Loss haute = le modele se trompe beaucoup
- Loss basse = le modele devine bien

Voyons comment ca marche avec un exemple :

In [ ]:
import math

# Imaginons que le modele predit les probabilites suivantes
# pour la lettre qui suit 'p' dans 'pikachu' :
prediction = {
    "i": 0.6,  # 60% -> bonne reponse !
    "a": 0.2,  # 20%
    "e": 0.15,  # 15%
    "o": 0.05,  # 5%
}

# La bonne reponse est 'i'
bonne_reponse = "i"

# La loss = -log(probabilite de la bonne reponse)
# Si le modele avait dit 100% pour 'i' : loss = 0 (parfait !)
# Si le modele avait dit 1% pour 'i' : loss = 4.6 (enorme !)
loss = -math.log(prediction[bonne_reponse])
print(
    f"Le modele donnait {prediction[bonne_reponse]:.0%} de chance a '{bonne_reponse}'"
)
print(f"Loss = {loss:.2f}")
print()

# Comparons avec une mauvaise prediction
mauvaise_prediction = {"i": 0.05, "a": 0.7, "e": 0.2, "o": 0.05}
loss_mauvaise = -math.log(mauvaise_prediction[bonne_reponse])
print(
    f"Si le modele n'avait donne que {mauvaise_prediction[bonne_reponse]:.0%} a '{bonne_reponse}'..."
)
print(f"Loss = {loss_mauvaise:.2f}  (beaucoup plus haut = beaucoup plus faux)")

A toi d'experimenter avec la loss :

In [ ]:
exercice(
    1,
    "Joue avec la perte",
    "Change <code>ma_prediction</code> ci-dessous (essaie 0.9 ou 0.01), puis <b>Shift + Entree</b>.",
    "Plus ta prediction est loin de la bonne reponse, plus la loss monte.",
)

# ╔══════════════════════════════════════╗
# ║  A TOI DE JOUER !                    ║
# ╠══════════════════════════════════════╣

ma_prediction = 0.6  # <-- Change cette valeur ! Essaie 0.9 ou 0.01

# ╚══════════════════════════════════════╝

ma_loss = -math.log(ma_prediction)
print(f"Si le modele donne {ma_prediction:.0%} de chance a la bonne reponse :")
print(f"  Loss = {ma_loss:.2f}")
if ma_prediction > 0.8:
    print("  -> Tres bien ! Le modele est confiant et a raison.")
elif ma_prediction < 0.1:
    print("  -> Enorme loss ! Le modele s'est beaucoup trompe.")
else:
    print("  -> Moyen. Le modele peut encore s'ameliorer.")

verifier(
    1,
    ma_prediction != 0.6,
    f"Bien joue ! Avec {ma_prediction:.0%}, la loss est {ma_loss:.2f}.",
    "Change ma_prediction pour une autre valeur, par exemple 0.9 ou 0.01.",
)

---
## Etape 2 : Les poids du modele

Un modele = une collection de **nombres** (les "poids").
Ces nombres determinent les predictions. **Entrainer = trouver les bons nombres.**

On va creer une grille 27x27 (26 lettres + le point) ou chaque case
contient un score. Au debut, les scores sont aleatoires :

In [ ]:
import math
import random

# Notre alphabet : 26 lettres + le point (debut/fin)
alphabet = list("abcdefghijklmnopqrstuvwxyz.")
print(f"Alphabet : {len(alphabet)} symboles")

# Scores aleatoires (les "poids" du modele)
random.seed(42)
poids = {}
for a in alphabet:
    poids[a] = {}
    for b in alphabet:
        poids[a][b] = random.uniform(-1, 1)  # nombre entre -1 et 1

print(f"Nombre de poids : {len(alphabet)} x {len(alphabet)} = {len(alphabet) ** 2}")


def calculer_probas(poids, lettre):
    """Transforme les scores en probabilites (softmax)."""
    scores = poids[lettre]
    # L'exponentielle rend tous les scores positifs
    exps = {b: math.exp(scores[b]) for b in scores}
    total = sum(exps.values())  # on divise par le total
    return {b: exps[b] / total for b in scores}


# Au debut, les probas sont quasi uniformes (le modele devine au hasard)
p = calculer_probas(poids, ".")
lettres_debut = sorted(p.items(), key=lambda x: -x[1])[:5]
print()
print("Au debut, le modele pense que les Pokemon commencent par :")
for lettre, prob in lettres_debut:
    print(f"  '{lettre}' : {prob:.1%}")
print("\n  -> C'est n'importe quoi ! Il faut l'entrainer.")

---
## Etape 3 : Entrainement

L'algorithme est simple :
1. Prendre un nom de Pokemon d'entrainement
2. Le modele fait sa prediction
3. On calcule la loss (l'erreur)
4. On **ajuste les poids** pour reduire la loss
5. Recommencer

L'etape 4 s'appelle la **descente de gradient**. C'est comme ajuster ton
tir au panier un petit peu a chaque essai.

L'entrainement prend ~2-3 secondes :

In [ ]:
# (c) Nintendo / Creatures Inc. / GAME FREAK inc. -- usage educatif
# Les 20 Pokemon d'entrainement
pokemons = [
    "arcanin",
    "bulbizarre",
    "carapuce",
    "dracaufeu",
    "ectoplasma",
    "evoli",
    "felinferno",
    "gardevoir",
    "goupix",
    "lokhlass",
    "lucario",
    "metamorph",
    "mewtwo",
    "noctali",
    "pikachu",
    "rondoudou",
    "ronflex",
    "salameche",
    "togepi",
    "voltali",
]

# Parametres d'entrainement
vitesse = 0.1  # de combien on ajuste a chaque fois
nb_epochs = 50  # nombre de passages sur les donnees

print("Entrainement (~2-3 secondes)...")
print()

_historique_loss = []  # pour la courbe

for epoch in range(nb_epochs):
    loss_totale = 0
    nb = 0

    for pokemon in pokemons:
        mot = "." + pokemon + "."  # ex: ".pikachu."
        for i in range(len(mot) - 1):
            lettre = mot[i]  # lettre courante
            cible = mot[i + 1]  # lettre a predire

            # 1. Prediction : probas pour chaque lettre suivante
            probas = calculer_probas(poids, lettre)

            # 2. Loss : a quel point on s'est trompe
            loss_totale += -math.log(probas[cible] + 1e-10)
            nb += 1

            # 3. Ajuster les poids (gradient simplifie)
            for b in alphabet:
                if b == cible:
                    # Bonne reponse : augmenter son score
                    poids[lettre][b] += vitesse * (1 - probas[b])
                else:
                    # Mauvaises reponses : baisser leur score
                    poids[lettre][b] -= vitesse * probas[b]

    _historique_loss.append(loss_totale / nb)  # loss moyenne

    if epoch % 10 == 0:
        print(f"  Epoch {epoch:2d} | Loss moyenne : {loss_totale / nb:.3f}")

print(f"  Epoch {epoch:2d} | Loss moyenne : {loss_totale / nb:.3f}")
print()
print("La loss baisse = le modele s'ameliore !")

# Courbe de loss animee
afficher_evolution_loss(
    _historique_loss, titre="Evolution de la loss pendant l'entrainement"
)

**La loss a baisse ! Qu'est-ce que ca veut dire ?**

Au debut, la loss etait haute (le modele devinait au hasard).
A la fin, elle est beaucoup plus basse (le modele a appris les
paires de lettres frequentes dans les noms de Pokemon).

---

In [ ]:
exercice(
    2,
    "Compare les vitesses",
    "Change <code>vitesse_test</code> ci-dessous (essaie 0.01, 0.1, 0.5, 2.0).",
    "Une vitesse trop grande fait exploser la loss, trop petite la fait stagner.",
)

# ╔══════════════════════════════════════╗
# ║  A TOI DE JOUER !                    ║
# ╠══════════════════════════════════════╣

vitesse_test = 0.5  # <-- Change cette valeur ! Essaie 0.01, 0.1, 0.5, 2.0

# ╚══════════════════════════════════════╝

# On repart de zero avec des poids aleatoires
random.seed(123)
poids_test = {}
for a in alphabet:
    poids_test[a] = {}
    for b in alphabet:
        poids_test[a][b] = random.uniform(-1, 1)

# Mini-entrainement de 5 epochs avec cette vitesse
for epoch in range(5):
    loss_t = 0
    nb_t = 0
    for pokemon in pokemons:
        mot = "." + pokemon + "."
        for i in range(len(mot) - 1):
            probas_t = calculer_probas(poids_test, mot[i])
            loss_t += -math.log(probas_t[mot[i + 1]] + 1e-10)
            nb_t += 1
            for b in alphabet:
                if b == mot[i + 1]:
                    poids_test[mot[i]][b] += vitesse_test * (1 - probas_t[b])
                else:
                    poids_test[mot[i]][b] -= vitesse_test * probas_t[b]
    print(f"  Epoch {epoch} | vitesse={vitesse_test} | Loss : {loss_t / nb_t:.3f}")

verifier(
    2,
    vitesse_test != 0.5,
    f"Bien ! Tu as teste la vitesse {vitesse_test}.",
    "Change vitesse_test pour une autre valeur, par exemple 0.01 ou 2.0.",
)

---
## Que sait le modele maintenant ?

Voyons si l'entrainement a porte ses fruits :

In [ ]:
# Apres entrainement, quelles lettres commencent les noms ?
p = calculer_probas(poids, ".")
lettres_debut = sorted(p.items(), key=lambda x: -x[1])[:5]
print("Apres entrainement, les Pokemon commencent par :")
for lettre, prob in lettres_debut:
    print(f"  '{lettre}' : {prob:.1%}")
print()
print("C'est plus logique ! (c, m, g, e, r sont des debuts courants)")

### Generons des noms de Pokemon !

Le modele peut maintenant inventer des noms en se basant
sur ce qu'il a appris :

In [ ]:
def generer(poids, n=10):
    """Genere n noms de Pokemon avec le modele entraine."""
    resultats = []
    for _ in range(n):
        pokemon = ""
        lettre = "."
        for _ in range(20):  # max 20 lettres
            p = calculer_probas(poids, lettre)
            choix = list(p.keys())  # lettres possibles
            probs = list(p.values())  # leurs probabilites
            lettre = random.choices(choix, weights=probs, k=1)[0]
            if lettre == ".":  # fin du nom
                break
            pokemon += lettre
        if pokemon:
            resultats.append(pokemon.capitalize())
    return resultats


print("Noms de Pokemon inventes apres entrainement :")
for p in generer(poids, 10):
    print(f"  {p}")

> **Coup de pouce** (si tu es bloque)
>
> Rappel : `generer(poids, 50)` genere 50 noms.
> Change juste le nombre dans les parentheses !

In [ ]:
exercice(
    3,
    "Genere des Pokemon",
    "Change <code>nombre</code> ci-dessous (essaie 50).",
    "Certains noms vont ressembler a de vrais Pokemon !",
)

# ╔══════════════════════════════════════╗
# ║  A TOI DE JOUER !                    ║
# ╠══════════════════════════════════════╣

nombre = 10  # <-- Mets 50 ici !

# ╚══════════════════════════════════════╝

print(f"Generation de {nombre} Pokemon :")
print()
for i, nom in enumerate(generer(poids, nombre)):
    print(f"  {i + 1}. {nom}")

verifier(
    3,
    nombre != 10,
    f"Genial ! Tu as genere {nombre} Pokemon.",
    "Change nombre pour une autre valeur, par exemple 50.",
)

**Compare avec la lecon 1 : les noms sont-ils meilleurs ?**

Dans la lecon 1, on comptait simplement les paires. Ici, le modele
a **appris** par lui-meme les meilleures probabilites en reduisant la loss.
Le resultat est similaire, mais la methode est celle de toutes les IA modernes !

---

> **Defi** (pour aller plus loin)
>
> Ecris un code qui entraine avec `vitesse=0.001` puis `vitesse=0.5`.
> Laquelle est meilleure ? Pourquoi ?

## Ce qu'on a appris

- La **loss** mesure a quel point le modele se trompe
- Les **poids** sont les nombres que le modele ajuste pour apprendre
- L'**entrainement** = ajuster les poids pour reduire la loss, encore et encore
- Meme un modele simple s'ameliore avec l'entrainement !

### Limite

Notre modele ne regarde encore que **1 lettre en arriere**.
Dans la prochaine lecon, on va lui donner une **memoire** pour qu'il
se souvienne de plusieurs lettres a la fois.

---
*Prochaine lecon : [03 - La memoire du modele](03_la_memoire_du_modele.ipynb)*

---

### Sources (ISO 42001)

- **Cross-entropy loss et descente de gradient** : [microgpt.py](https://gist.github.com/karpathy/8627fe009c40f57531cb18360106ce95) \u2014 Andrej Karpathy, lignes implementant le backward pass
- **Analogie du gradient comme correction** : [Video "Let's build GPT"](https://www.youtube.com/watch?v=kCc8FmEb1nY) \u2014 Andrej Karpathy (2023)
- **Visualisation de la descente de gradient** : [3Blue1Brown - Gradient descent](https://www.youtube.com/watch?v=IHZwWFHWa-w) \u2014 Grant Sanderson
- **Dataset Pokemon** : (c) Nintendo / Creatures Inc. / GAME FREAK inc., usage educatif. Source : [PokeAPI](https://pokeapi.co/)